# CVRP Notebook
This notebook is intended to run the top level code for the CVRP

In [1]:
import networkx as nx

from model import VRP, create_constraints, output_variables
from pricer import VRPPricer
from parse import parse

from matplotlib import pyplot as plt

In [2]:
# Test instance E-n22-k4 provided by parser
# G = parse("Instances/E/E-n22-k4.vrp")
# G = parse("Instances/E/E-n23-k3.vrp")
# G = parse("Instances/E/E-n30-k3.vrp")
# G = parse("Instances/E/E-n33-k4.vrp")
# G = parse("Instances/E/E-n51-k5.vrp")
G = parse("Instances/E/E-n76-k14.vrp")
# G = parse("Instances/E/E-n101-k14.vrp")
# G = parse("Instances/Uchoa/X-n101-k25.vrp")
# G.graph['min_trucks'] = 26
# G = parse("Instances/Uchoa/X-n256-k16.vrp")
# G.graph['min_trucks'] = 16
# G = parse("Instances/Uchoa/X-n129-k18.vrp")
# G.graph['min_trucks'] = 18

PARSE: Minimum number of trucks is 14


In [ ]:
model = VRP(G)

# Create pricer
pricer = VRPPricer()
pricer.init_data(G)
pricer.data['methods'] = ["ng8","cyc2","SPPRC"]
pricer.data['max_vars']= 10000
pricer.data['abort_early'] = False

model.includePricer(pricer, "pricer","does pricing")

create_constraints(model,G,heuristic_flag=True)

# model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)

PRICER_PY: The maximal path length is 52
PY PRICING: The neighborhood has been fixed to 8 neighbors.
PRICER_C: Graph data successfully copied to C.
presolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 89 variables (0 bin, 0 int, 0 impl, 89 cont) and 76 constraints
     76 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     1 |     0 |    31 |     - |    LP  |   0 |  89 |  76 |  76 |   0 |  0 |   0 |   0 |      --      | 1.021000e+03 |    Inf | unknown
  2.5s|     1 |     0 |    75 |     - |    21M |   0 |6890 |  76 |  76 |   0 |  0 |   0 |   0 |-1.849000e+03 | 1.021000e+03 |    Inf | unknown


In [ ]:
plt.figure(figsize=(18,6))
xmax = 0
for method in pricer.data['methods']:
    upper_bounds = [x for x,y in pricer.data['bounds'][method]]
    lower_bounds = [y for x,y in pricer.data['bounds'][method]]
    plt.plot(upper_bounds, label=f'{method} UB')
    plt.plot(lower_bounds, label=f'{method} LB')
    if upper_bounds[0] > xmax:
        xmax = upper_bounds[0]
plt.ylim((-xmax/10,xmax*1.1))

T = nx.minimum_spanning_tree(G)
mst = T.size(weight='weight') + min(G.edges().data('weight'),key=lambda edge: edge[2])[2]
mst = [mst for i in range(len(upper_bounds))]
plt.plot(mst, label="MST Bound")

solval = model.getObjVal()
sol = [solval for i in range(len(upper_bounds))]
plt.plot(sol,label="Solution value")

plt.legend(loc='upper right')

plt.show()

In [ ]:
output_variables(model, pricer)